# Credit Risk Resampling Techniques

In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [34]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [40]:
columns = [
    "Salary cap", "Season", "Wins", "Losses","PCT" "Postseason", "FG",
    "FGA", "FG%", "3PM", "3P", "3PA", "3P%","2P", "2PA", "2P%", "FT", "FTA", "FT%", "ORB", "DRB", "TRB", 
    "AST", "TOV", "STL", "BLK", "BLKA", "PF", "PTS"
]

# columns = [
#     "loan_amnt", "int_rate", "installment", "home_ownership",
#     "annual_inc", "verification_status", "issue_d", "loan_status",
#     "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
#     "open_acc", "pub_rec", "revol_bal", "total_acc",
#     "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
#     "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
#     "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
#     "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
#     "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
#     "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
#     "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
#     "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
#     "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
#     "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
#     "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
#     "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
#     "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
#     "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
#     "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
#     "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
#     "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
#     "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
# ]

target = ["Postseason"]

In [41]:
# Load the data
file_path = Path('Data/NBA_Database.csv')
df = pd.read_csv(file_path)[:-2]
# df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
# postszn_mask = df["Postszn"] != 'Playoffs'
# df = df.loc[postszn_mask]

# # Convert the target column values to low_risk and high_risk based on their values
# x = {'Current': 'low_risk'}   
# df = df.replace(x)

# x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
# df = df.replace(x)

# df.reset_index(inplace=True, drop=True)


In [42]:
print(df.dtypes)

Salary cap      int64
Wins            int64
Losses          int64
PCT           float64
Postseason     object
FG            float64
FGA           float64
FG%           float64
3P            float64
3PA           float64
3P%           float64
2P            float64
2PA           float64
2P%           float64
FT            float64
FTA           float64
FT%           float64
ORB           float64
DRB           float64
TRB           float64
AST           float64
STL           float64
BLK           float64
TOV           float64
PF            float64
PTS           float64
dtype: object


# Split the Data into Training and Testing

In [67]:
# Create our features
X = pd.get_dummies(df, columns= ["Salary cap",
                                "PCT",
                                "FG%",
                                "3P%",
                                "2P%",
                                "FT%",
                                "TRB",
                                "AST",
                                "STL",
                                "BLK",
                                "TOV",
                                "PTS"]).drop('Postseason', axis=1)

# Create our target

y = df['Postseason']
X.head()

,Wins,Losses,FG,FGA,3P,3PA,2P,2PA,FT,FTA,...,PTS_116.3,PTS_116.4,PTS_116.6,PTS_117.0,PTS_117.7,PTS_117.8,PTS_118.0,PTS_118.6,PTS_118.7,PTS_120.1
0,53,29,40.5,86.4,14.3,39.4,26.2,47.0,15.6,20.3,...,0,0,0,0,0,0,0,0,0,0
1,44,38,42.0,88.4,11.5,31.7,30.5,56.7,17.5,21.7,...,0,0,0,0,0,0,0,0,0,0
2,42,40,40.1,87.4,12.8,34.2,27.3,53.3,15.5,19.6,...,0,0,0,0,0,0,0,0,0,0
3,33,49,41.6,88.8,12.0,34.5,29.7,54.3,16.8,23.0,...,0,0,0,0,0,0,0,0,0,0
4,51,31,41.8,89.4,14.1,38.4,27.8,51.0,17.8,22.9,...,0,0,0,0,0,0,0,0,0,0


In [68]:
X.describe()

,Wins,Losses,FG,FGA,3P,3PA,2P,2PA,FT,FTA,...,PTS_116.3,PTS_116.4,PTS_116.6,PTS_117.0,PTS_117.7,PTS_117.8,PTS_118.0,PTS_118.6,PTS_118.7,PTS_120.1
count,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,...,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000
mean,39.054054,38.608108,40.714189,88.120270,11.833108,32.962838,28.883108,55.154054,17.268243,22.370946,...,0.006757,0.006757,0.006757,0.006757,0.006757,0.006757,0.006757,0.006757,0.006757,0.006757
std,11.675769,11.160935,1.608608,2.282795,1.681245,4.430495,2.141599,4.466982,1.529352,1.984986,...,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199,0.082199
min,15.000000,17.000000,36.700000,82.800000,8.000000,22.500000,23.100000,41.900000,12.300000,16.700000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,31.000000,30.000000,39.400000,86.475000,10.675000,29.975000,27.375000,52.475000,16.275000,20.975000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,41.500000,38.000000,40.700000,88.100000,11.600000,32.700000,29.000000,55.300000,17.200000,22.300000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,48.000000,46.000000,41.900000,89.625000,12.925000,35.950000,30.525000,58.300000,18.100000,23.400000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,65.000000,65.000000,44.700000,94.400000,16.700000,45.400000,33.400000,64.400000,22.600000,28.500000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [69]:
# Check the balance of our target values
y.value_counts()

Playoffs           79
Missed Playoffs    69
Name: Postseason, dtype: int64

In [70]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({'Missed Playoffs': 50, 'Playoffs': 61})

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [71]:
# Resample the training data with the RandomOversampler

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'Missed Playoffs': 61, 'Playoffs': 61})

In [72]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [73]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

1.0

In [74]:
# Display the confusion matrix

from sklearn.metrics import confusion_matrix

cm1 = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm1_df = pd.DataFrame(cm1, index=["Actual Missed Playoffs", "Actual Playoffs"], columns=["Predicted Missed Playoffs", "Predicted Playoffs"])
cm1_df

,Predicted Missed Playoffs,Predicted Playoffs
Actual Missed Playoffs,19,0
Actual Playoffs,0,18


In [75]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                       pre       rec       spe        f1       geo       iba       sup

Missed Playoffs       1.00      1.00      1.00      1.00      1.00      1.00        19
       Playoffs       1.00      1.00      1.00      1.00      1.00      1.00        18

    avg / total       1.00      1.00      1.00      1.00      1.00      1.00        37



### SMOTE Oversampling

In [52]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
Counter(y_resampled)

Counter({'Missed Playoffs': 61, 'Playoffs': 61})

In [53]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [54]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

1.0

In [55]:
# Display the confusion matrix
# Display the confusion matrix
cm2 = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm2_df = pd.DataFrame(cm2, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
cm2_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,19,0
Actual low_risk,0,18


In [56]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                       pre       rec       spe        f1       geo       iba       sup

Missed Playoffs       1.00      1.00      1.00      1.00      1.00      1.00        19
       Playoffs       1.00      1.00      1.00      1.00      1.00      1.00        18

    avg / total       1.00      1.00      1.00      1.00      1.00      1.00        37



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [57]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete

from imblearn.under_sampling import ClusterCentroids
# Instantiate
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'Missed Playoffs': 50, 'Playoffs': 50})

In [58]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [59]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

1.0

In [60]:
# Display the confusion matrix
cm3 = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm3_df = pd.DataFrame(cm3, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
cm3_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,19,0
Actual low_risk,0,18


In [61]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                       pre       rec       spe        f1       geo       iba       sup

Missed Playoffs       1.00      1.00      1.00      1.00      1.00      1.00        19
       Playoffs       1.00      1.00      1.00      1.00      1.00      1.00        18

    avg / total       1.00      1.00      1.00      1.00      1.00      1.00        37



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [62]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'Missed Playoffs': 50, 'Playoffs': 50})

In [63]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=78)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=78)

In [64]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

1.0

In [65]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(cm, index=["Actual high_risk", "Actual low_risk"], columns=["Predicted high_risk", "Predicted low_risk"])
cm_df

,Predicted high_risk,Predicted low_risk
Actual high_risk,19,0
Actual low_risk,0,18


In [66]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                       pre       rec       spe        f1       geo       iba       sup

Missed Playoffs       1.00      1.00      1.00      1.00      1.00      1.00        19
       Playoffs       1.00      1.00      1.00      1.00      1.00      1.00        18

    avg / total       1.00      1.00      1.00      1.00      1.00      1.00        37

